In [30]:
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import hashlib

watermark_information = "00000100"

g = 8
seed = 10000
secret_key_1 = "123"
secret_key_2 = "456"


dataset='covertype'


original_file = '../dataset/covtype_with_key.subset.data'
origin = pd.read_csv(original_file)

np.random.seed(seed)

cover_types = origin['Cover_Type'].unique()
cover_types.sort()

def hash_mod(key, mod_value, secret_key):
    # 使用 SHA-256 生成哈希
    combined = f"{secret_key}{key}"
    hash_value = int(hashlib.sha256(combined.encode()).hexdigest(), 16)  # 将哈希转换为整数
    return hash_value % mod_value  # 对结果取模

columns_of_interest = ['Elevation', 'Aspect']  # 可根据需求更改
origin[columns_of_interest] = origin[columns_of_interest].fillna(0)  # 填充NA为0

# 提取前两位非零数字的函数
def first_two_digits(x):
    if x == 0:
        return "00"
    digits = str(x).lstrip('0.').replace('.', '')  # 去掉前导0和小数点
    if len(digits) == 1:  # 如果只有一位数字
        return digits + "0"  # 补0
    return digits[:2]  # 返回前两位 


np.random.seed(seed)
shuffled_cover_types = list(cover_types)
shuffled_cover_types = sorted(shuffled_cover_types)
np.random.shuffle(shuffled_cover_types)
half_size = len(shuffled_cover_types) // 2
green_domain = shuffled_cover_types[:half_size]
red_domain = shuffled_cover_types[half_size:]
# print(green_domain, red_domain)

for idx in range(len(origin)):
    selected_data = origin.loc[idx, columns_of_interest]
    first_two_digits_data = selected_data.apply(first_two_digits)
    composite_numbers = ''.join(first_two_digits_data.values)
    
    if(watermark_information[hash_mod(composite_numbers, len(watermark_information), secret_key_1)] == '1'):
        if(hash_mod(composite_numbers, g, secret_key_2) == 0):
            if origin.loc[idx, 'Cover_Type'] in red_domain:
                perturb_value = np.random.choice(green_domain)
                origin.loc[idx, 'Cover_Type'] = perturb_value   
         



results = {
    'watermarked_data': origin
}

decimal = int(watermark_information, 2)

np.save(f"different_version_datasets/{dataset}-{decimal}.npy", results)






In [29]:
# detect
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import hashlib


g = 8
seed = 10000
secret_key_1 = "123"
secret_key_2 = "456"
threshold = 1.96

dataset='covertype'


original_file = '../dataset/covtype_with_key.subset.data'
origin = pd.read_csv(original_file)

np.random.seed(seed)

cover_types = origin['Cover_Type'].unique()
cover_types.sort()

def hash_mod(key, mod_value, secret_key):
    # 使用 SHA-256 生成哈希
    combined = f"{secret_key}{key}"
    hash_value = int(hashlib.sha256(combined.encode()).hexdigest(), 16)  # 将哈希转换为整数
    return hash_value % mod_value  # 对结果取模

columns_of_interest = ['Elevation', 'Aspect']  # 可根据需求更改
origin[columns_of_interest] = origin[columns_of_interest].fillna(0)  # 填充NA为0

# 提取前两位非零数字的函数
def first_two_digits(x):
    if x == 0:
        return "00"
    digits = str(x).lstrip('0.').replace('.', '')  # 去掉前导0和小数点
    if len(digits) == 1:  # 如果只有一位数字
        return digits + "0"  # 补0
    return digits[:2]  # 返回前两位


detected_watermark_information = ""


loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
watermarked_data = loaded_results['watermarked_data']
# watermarked_data = pd.read_csv("../dataset/covtype_with_key.subset.data")

watermarked_data[columns_of_interest] = watermarked_data[columns_of_interest].fillna(0)  # 填充NA为0

cover_types = watermarked_data['Cover_Type'].unique()
cover_types.sort()  

np.random.seed(seed)
shuffled_cover_types = list(cover_types)
shuffled_cover_types = sorted(shuffled_cover_types)
np.random.shuffle(shuffled_cover_types)
half_size = len(shuffled_cover_types) // 2
green_domain = shuffled_cover_types[:half_size]
red_domain = shuffled_cover_types[half_size:]
# print(green_domain, red_domain)

green_cells = np.zeros(len(watermark_information))
n_cells = np.zeros(len(watermark_information))
z_scores = np.zeros(len(watermark_information))

for idx in range(len(watermarked_data)):

    selected_data = watermarked_data.loc[idx, columns_of_interest]
    first_two_digits_data = selected_data.apply(first_two_digits)
    composite_numbers = ''.join(first_two_digits_data.values)
    
    w_index = hash_mod(composite_numbers, len(watermark_information), secret_key_1)
    
    if(hash_mod(composite_numbers, g, secret_key_2) == 0): 
        n_cells[w_index] += 1    
        if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
            green_cells[w_index] += 1
        # else:
        #     if(watermark_information[w_index] == '1'):
        #         print(watermarked_data.loc[idx, 'Cover_Type'])
            
for idx in range(len(watermark_information)):
    if n_cells[idx] != 0:
        z_scores[idx] = (green_cells[idx] - n_cells[idx]/2 ) / math.sqrt(n_cells[idx]/4)
    else:
        z_scores[idx] = 0
    
    # print(n_cells[idx])
    # print(green_cells[idx])
    print(z_scores[idx])
    # print('---')    


for idx in range(len(z_scores)):
    if z_scores[idx] > threshold:
        detected_watermark_information += '1'
    else:
        detected_watermark_information += '0'
        
print(detected_watermark_information)





-5.678573818125704
-1.049344364594206
-2.1908902300206643
-0.8305395344999218
-0.19802950859533489
17.492855684535904
-0.8250286473253902
1.1895773785772161
00000100
